In [259]:
import os
import email
import base64
import numpy as np
import sklearn 

from bs4 import BeautifulSoup as soup

In [260]:
def extract(file_content):
    #d=file_content.split("Subject:")
    #print d[1]
    b = email.message_from_string(file_content)
    if b.is_multipart():
        a=''
        for payload in b.get_payload():
        # if payload.is_multipart(): ...
             a.join(str(payload.get_payload()))
             return a
    else:
        return b.get_payload()
   



In [261]:
def vectorize(body):
    body=body.lower()
    a="hello"
    b="how"
    c="are"
    d="you"
    l=[a,b,c,d]
    a=[0,0,0,0]
    for i,x in enumerate(l):
        if x in body:
            a[i]=1
    return a
    

In [262]:
def create(folder_name):
    
    file_list=os.listdir(folder_name)
    vector=[]
       
    for i in file_list:
        path=os.path.join(folder_name,i)
        f=open(path,'r').read()
        body=extract(f) 
        vector.append(vectorize(body))
    
    return vector
    
    
    
    

In [263]:
def convert(arr):
   
    test=[]
    for i in arr:
        test.append(i[0])
    test=np.asarray(test)
    arr=test
    return arr

In [264]:
spam_vector=np.asarray(create("spam"))
spam_label=np.ones([len(spam_vector),1])

In [265]:
ham_vector=np.asarray(create("easy_ham"))
ham_label=np.zeros([len(ham_vector),1])

type(ham_vector[3])

numpy.ndarray

In [266]:
hard_ham_vector=np.asarray(create("hard_ham"))
hard_ham_label=np.zeros([len(hard_ham_vector),4])

email_data=np.concatenate((spam_vector,ham_vector),axis=0)
#email_data=np.concatenate((spam_vector,ham_vector,hard_ham_vector),axis=0)

email_label=np.concatenate((spam_label,ham_label),axis=0)



In [267]:
from sklearn.model_selection import train_test_split

train_set,test_set=train_test_split(email_data,test_size=0.1,random_state=42)
train_label,test_label=train_test_split(email_label,test_size=0.1,random_state=42)

In [268]:
test_label=convert(test_label)
train_label=convert(train_label)




In [269]:
from sklearn.metrics import accuracy_score
accuracy_score(test_label,clf.predict(test_set))


0.79411764705882348

In [287]:
def train(lf):
    print(lf.fit(train_set,train_label.ravel()))
    return accuracy_score(test_label,lf.predict(test_set))
    

In [285]:
from sklearn import tree
clf=tree.DecisionTreeClassifier()
print(clf.fit(train_set,train_label))

print(train(clf))

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            presort=False, random_state=None, splitter='best')
0.794117647059


In [286]:
from sklearn.neighbors import KNeighborsClassifier
knn=KNeighborsClassifier()
print(train(knn))

0.794117647059


In [273]:
from sklearn.naive_bayes import GaussianNB as GNB
nbc=GNB()
print(train(nbc))

0.794117647059


In [274]:
from sklearn import svm
svc=svm.SVC()
print(train(svc))


0.794117647059


In [275]:
from sklearn.model_selection import GridSearchCV


In [281]:
from sklearn.model_selection import cross_val_score
scores = cross_val_score(clf, train_set, train_label,
scoring="neg_mean_squared_error", cv=10)
rmse_scores = np.sqrt(-scores)
rmse_scores

array([ 0.4       ,  0.4       ,  0.4       ,  0.4       ,  0.4       ,
        0.4       ,  0.4       ,  0.4       ,  0.39687423,  0.39687423])

In [224]:
param_grid = [
{'n_estimators': [3, 10, 30], 'max_features': [2, 4, 6, 8]},
{'bootstrap': [False], 'n_estimators': [3, 10], 'max_features': [2, 3, 4]},
]

In [276]:
grid_search=GridSearchCV(clf,param_grid,cv=5,scoring="neg_mean_squared_error")

In [283]:
from sklearn.ensemble import RandomForestRegressor
forest_reg = RandomForestRegressor()
forest_reg.fit(train_set, train_label)


RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_split=1e-07, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           n_estimators=10, n_jobs=1, oob_score=False, random_state=None,
           verbose=0, warm_start=False)

In [299]:
print(train(svc))
clf=svc
clf

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)
0.794117647059


SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [303]:
param_grid = [
{'C':[0.01,0.5,0.1],'cache_size':[100,200,300],'degree':[1,2,3,4,5]},
{'shrinking':[False],'tol':[0,0.01,0.001,0.0001]},
]

In [304]:
grid_search=GridSearchCV(clf,param_grid,cv=5,scoring="neg_mean_squared_error")

In [305]:
grid_search.fit(train_set,train_label)

ValueError: eps <= 0